# Tratamento e limpeza de dados 

# Imports

In [1]:
import pandas as pd
import numpy as np

# Sobre os dados



### Contexto

A coleta de dados de aplicativos da Apple App Store é relativamente fácil devido à estrutura bem organizada de suas páginas, o que facilita a raspagem de dados. No entanto, a obtenção de dados do Google Play Store é mais desafiadora porque a loja utiliza técnicas modernas, como carregamento dinâmico de páginas com JQuery, que complicam o processo de raspagem.

Esse contexto explica as dificuldades em obter dados do Google Play Store, justificando a raridade desses datasets em comparação aos dados da Apple App Store.

### Objetivo 

Neste notebook, meu principal objetivo é realizar uma análise criteriosa do conjunto de dados, identificando e corrigindo possíveis inconsistências e anomalias. Este processo de limpeza e preparação dos dados é essencial para garantir que a posterior análise exploratória seja precisa e confiável, permitindo insights robustos e relevantes a partir dos dados tratados

### Fonte dos dados
Os arquivos do dataset foram obtidos do seguinte repositório do Kaggle:
https://www.kaggle.com/datasets/lava18/google-play-store-apps?resource=download&select=googleplaystore_user_reviews.csv


# Data Loading

In [2]:
path_ = '../dataset/googleplaystore.csv'

In [3]:
df = pd.read_csv(path_)

## Data exploration

In [4]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [8]:
df.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type',
       'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver'],
      dtype='object')

In [9]:
df.shape

(10841, 13)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


# Identificando inconsistências

## Valores duplicados

In [12]:
df.duplicated().sum()

483